# Demonstration workflow for *openQmin* (+ *openViewMin*) within a Jupyter notebook

First, let's define some basic global properties.

In [ ]:
# system size *before* MPI subdivision
whole_Lx = 52
whole_Ly = 40
whole_Lz = 62
S = 0.53 # uniaxial order for all sites
mpi_num_processes = 3 # set to 1 for non-MPI run

### Boundaries
Next, we'll create a *boundaryFile* (in this case describing a spherical colloid and a planar wall).

In [ ]:
import boundaryHelper as bh

bdyfilename = '../data/sphere_and_wall.txt'

sc = bh.Scene(whole_Lx, whole_Ly, whole_Lz) # make a Scene
anch1= bh.OrientedAnchoringCondition(strength=5.3, S0=0.53) # first anchoring condition
co = bh.SphericalColloid(anch1, ((whole_Lx-1)/2, (whole_Ly-1)/2, (whole_Lz-1)/2), 10) # colloid boundary object
anch2 = bh.DegeneratePlanarAnchoringCondition(strength=10, S0=0.53) # second anchoring condition
wall = bh.Wall(anch2, "x", 5) # wall boundary object, normal to x, positioned at x=5
sc.boundary_objects = [co, wall] # put objects in Scene
sc.to_file(bdyfilename) # write Scene to text file

### Initialization
Optionally, we can define an initial director field, which we save to a file for import by *openQmin* as *initialConfigurationFile*.

In [ ]:
import initHelper as ih

initfilename="../data/my_init_file" # file prefix for where to save output

nx_function_string = "cos(rho*2*pi/50)*cos(phi)" 
ny_function_string = "cos(rho*2*pi/50)*sin(phi)" 
nz_function_string = "sin(rho*2*pi/50)"

ih.create_init_state(
    whole_Lx, whole_Ly, whole_Lz, S, mpi_num_processes, initfilename, 
    nx_function_string, ny_function_string, nz_function_string
)

### Run
Now we can use *runHelper.py* to generate the command-line call to *openQmin* (with any changes we'd like to make to the default parameters) and then run it as a shell command from within Jupyter.

In [ ]:
import runHelper as rh

saveFile = '../data/demo_run' # choose a location and filename-prefix for this run's results

rh.mpi_num_processes = mpi_num_processes

rh.params['initialConfigurationFile'] = initfilename
rh.params['boundaryFile'] = bdyfilename
rh.params['saveFile'] = saveFile
rh.params['iterations'] = 500
rh.params['whole_Lx'] = whole_Lx
rh.params['whole_Ly'] = whole_Ly
rh.params['whole_Lz'] = whole_Lz

runcmd = rh.get_runcmd() # generate command-line string
!{runcmd} # run openQmin.out from shell command

### Visualization
Finally, we'll visualize the result using *openViewMin*. 

In [ ]:
import sys
sys.path.append('../../openviewmin') # replace with your own path to openViewMin's directory
import openViewMin

openViewMin.start_xvfb() # start a virtual frame buffer

In [ ]:
# load the openQmin result
import glob
result_files = glob.glob(saveFile + '_x*y*z*.txt')
ovm_plot = openViewMin.NematicPlot(result_files) 

While the full interactive mode of *openViewMin* can't be run inside a Jupyter notebook, we have a couple of options if we want to take a quick look at our results here. The first is an interactive 3D view without widgets. 

In [ ]:
# Optionally make changes using PyVista or openViewMin-specific methods
ovm_plot.update_actor('defects', filter_kwargs={"isosurfaces":[0.3]})
ovm_plot.set_lights_intensity(0.6)
# Use pythreejs to render in-notebook
ovm_plot.to_pythreejs()

The second option is to take a snapshot of the *openViewMin* plot's current state.

In [ ]:
ovm_plot.camera.position = (150,100,100) 
ovm_plot.set_lights_intensity(5.)

In [ ]:
import matplotlib.pyplot as plt

# Plot bitmap of openViewMin screenshot
fig = plt.figure(figsize=(10,10))
plt.imshow(ovm_plot.screenshot()) 
plt.show()
fig.savefig('demo_run_vis.png')